## Installs

In [ ]:
!pip install pykeen

## Imports

In [1]:
# To store the data
import pandas as pd

# To do linear algebra
import numpy as np

# To apply Knowledge Graphs
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline
from pykeen.models import predict

# To do train-test split for evaluation
from sklearn.model_selection import train_test_split

## Load Data

In [2]:
#Function to Extract users, movies and ratings from raw data
def extract_users_items_predictions(data_pd):
    users, movies = \
        [np.squeeze(arr) for arr in np.split(data_pd.Id.str.extract('r(\d+)_c(\d+)').values.astype(int) - 1, 2, axis=-1)]
    predictions = data_pd.Prediction.values
    return users, movies, predictions

#Load raw data
data_pd = pd.read_csv('../data/data_train.csv')

#Do Train-Test Split
train, val = train_test_split(data_pd, test_size=0.1,random_state=42)

#Extract users, movies and ratings from raw data
total_users, total_movies, total_pred = extract_users_items_predictions(data_pd)
train_users, train_movies, train_pred = extract_users_items_predictions(train)
val_users, val_movies, val_pred = extract_users_items_predictions(val)

#Store total data
ratings_dict_total = {'userID': total_users.astype(str),'rating': total_pred.astype(str),'movieID': total_movies.astype(str)}
df_total = pd.DataFrame(ratings_dict_total)

#Store train data
ratings_dict_train = {'userID': train_users.astype(str),'rating': train_pred.astype(str),'movieID': train_movies.astype(str)}
df_train = pd.DataFrame(ratings_dict_train)

#Store validation data
ratings_dict_test = {'userID': val_users,'rating': val_pred,'movieID': val_movies}
df_test = pd.DataFrame(ratings_dict_test)

# Post Process to Generate Prediction Matrix

In [3]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

# Predicts the score for each user,movie,rating combination 
# and then performs weighted averaging to get final rating for each user/movie
def kg_postprocess(result,number_of_users,number_of_movies):
    A = np.zeros((5,number_of_users, number_of_movies))
    B = np.zeros((number_of_users, number_of_movies))
    triples = []
    for i in range(number_of_users):
        for j in range(number_of_movies):
            for k in range(5):
                triples.append((str(i),str(k+1),str(j)))
    score_df = predict.predict_triples_df(model=result.model, triples=triples, triples_factory=result.training,batch_size=1024, )

    for index, row in score_df.iterrows():
        A[int(row['relation_label'])-1][int(row['head_label'])][int(row['tail_label'])] = float(row['score'])



    for i in range(number_of_users):
        for j in range(number_of_movies):
            B[i, j] = np.sum(softmax(A[:, i, :][:, j]) * (np.arange(5) + 1))

    return B

# Return root mean square error metric
def RMSE(x: np.ndarray, y: np.ndarray) -> float:
    return np.sqrt(np.average((x - y) ** 2))

# Evaluation and Parameter Tuning

In [4]:
#Define the Triples factory for Training
training_path = TriplesFactory.from_labeled_triples(df_train.values,
                                                    create_inverse_triples=False,
                                                    entity_to_id=None,relation_to_id=None,
                                                    compact_id=None,filter_out_candidate_inverse_relations=True,
                                                    metadata=None,)

testing_path = TriplesFactory.from_labeled_triples(df_train.head(1).values, 
                                                   create_inverse_triples=False, 
                                                   entity_to_id=None,relation_to_id=None, 
                                                   compact_id=None,filter_out_candidate_inverse_relations=True, 
                                                   metadata=None, )

#Validation on Different types of Models/Embedding sizes
def validation(training_path,df_test):
    model = ['TransE', 'DistMult', 'MuRE']
    embedding_dim = [50,100,150] 
    best_rmse = 2
    best_model = 0
    best_embedding_dim = 0
    
    for m in model:
        for e in embedding_dim:
            result = pipeline(training=training_path,
                  testing=testing_path,
                  model=m,
                  model_kwargs=dict(embedding_dim=e),
                  training_kwargs=dict(num_epochs=100, batch_size=256),
                  random_seed = 1234,
                 )
            pred_matrix = kg_postprocess(result,10000,1000)
            test_predictions = pred_matrix[df_test['userID'], df_test['movieID']]
            rmse = RMSE(df_test['rating'],test_predictions)
            if rmse < best_rmse:
                best_rmse = rmse
                best_model = m
                best_embedding_dim = e
                print(best_rmse)
    
    return best_rmse, best_model, best_embedding_dim

best_rmse,best_model,best_embedding_dim = validation(training_path,df_test)

No cuda devices were available. The model runs on CPU
/Users/samyakjain/miniforge3/lib/python3.9/site-packages/pykeen/nn/representation.py:369: UserWarning: Directly use Embedding.max_id instead of num_embeddings.
  warnings.warn(f"Directly use {self.__class__.__name__}.max_id instead of num_embeddings.")
/Users/samyakjain/miniforge3/lib/python3.9/site-packages/pykeen/nn/representation.py:375: UserWarning: Directly use Embedding.shape instead of num_embeddings.
  warnings.warn(f"Directly use {self.__class__.__name__}.shape instead of num_embeddings.")


Training epochs on cpu:   0%|          | 0/100 [00:00<?, ?epoch/s]

Training batches on cpu:   0%|          | 0/4138 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/4138 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/4138 [00:00<?, ?batch/s]

...

Training batches on cpu:   0%|          | 0/4138 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/4138 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Currently automatic memory optimization only supports GPUs, but you're using a CPU. Therefore, the batch_size will be set to the default value.
INFO:pykeen.evaluation.evaluator:No evaluation batch_size provided. Setting batch_size to '32'.


Evaluating on cpu:   0%|          | 0.00/1.00 [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 1.44s seconds


In [5]:
print("Validation RMSE using model is " + str(best_rmse))

Validation RMSE using model is 1.2992216693378587


## Train model on Total Data

In [6]:
#Define the Triples factory for Training
training_path = TriplesFactory.from_labeled_triples(df_total.values,
                                                    create_inverse_triples=False,
                                                    entity_to_id=None,relation_to_id=None,
                                                    compact_id=None,filter_out_candidate_inverse_relations=True,
                                                    metadata=None,)
testing_path = TriplesFactory.from_labeled_triples(df_total.head(1).values, 
                                                   create_inverse_triples=False, 
                                                   entity_to_id=None,relation_to_id=None, 
                                                   compact_id=None,filter_out_candidate_inverse_relations=True, 
                                                   metadata=None, )

#Train the best model
result = pipeline(training=training_path,
                  testing=testing_path,
                  model=best_model,
                  model_kwargs=dict(embedding_dim=best_embedding_dim),
                  training_kwargs=dict(num_epochs=100, batch_size=256),
                 )

#Generate Prediction Matrix
pred_matrix = kg_postprocess(result,10000,1000)

INFO:pykeen.pipeline.api:Using device: None


Training epochs on cpu:   0%|          | 0/100 [00:00<?, ?epoch/s]

Training batches on cpu:   0%|          | 0/4598 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/4598 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/4598 [00:00<?, ?batch/s]

...

Training batches on cpu:   0%|          | 0/4598 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/4598 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Currently automatic memory optimization only supports GPUs, but you're using a CPU. Therefore, the batch_size will be set to the default value.
INFO:pykeen.evaluation.evaluator:No evaluation batch_size provided. Setting batch_size to '32'.


Evaluating on cpu:   0%|          | 0.00/1.00 [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 1.10s seconds


## Generate Submission File

In [8]:
#Load Submission File
sub_pd = pd.read_csv('../data/sampleSubmission.csv')
sub_users, sub_movies, sub_pred = extract_users_items_predictions(sub_pd)
sub_test_ratings_dict = {'userID': sub_users,'movieID': sub_movies,'rating': sub_pred}
sub_df = pd.DataFrame(sub_test_ratings_dict)
sub_user_data = sub_df['userID']
sub_movie_data = sub_df['movieID']

#Sample Predictions
pred = []
for i in range(sub_user_data.shape[0]):
    pred.append(pred_matrix[sub_user_data[i]][sub_movie_data[i]])

#Generate Submission csvs
sub_pd['Prediction'] = pred
sub_pd.set_index("Id", inplace = True)
sub_pd.to_csv('submission_knowledge_graph.csv')